In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" ||
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/airbnb-recruiting-new-user-bookings/train_users_2.csv.zip")
test = pd.read_csv("/kaggle/input/airbnb-recruiting-new-user-bookings/test_users.csv.zip")

In [ ]:
session = pd.read_csv("/kaggle/input/airbnb-recruiting-new-user-bookings/sessions.csv.zip")

In [ ]:
display(train,test,session)

In [ ]:
train["first_browser"].unique()

In [ ]:
session = session.rename({"user_id":"id"}, axis=1) #axis무조건 넣어야 함
id_count = session.groupby("id")["action_type"].count()
id_sec = session.groupby("id")["secs_elapsed"].mean()
id_nunique = session.groupby("id")["action","action_type","action_detail","device_type"].nunique()
id_nunique.columns = ["action_nunique","action_type_nunique","detail_nunique",'device_nunique']

In [ ]:
etc = session["action"].value_counts()[session["action"].value_counts() < 10000].index #5000개이하인 것들을 etc로 저장

In [ ]:
session["action"] = session["action"].apply(lambda x:"etc" if x in etc else x)

In [ ]:
session["action"].value_counts()

In [ ]:
session_action = pd.get_dummies(session["action"]) #카테고리인 컬럼을 원핫인코딩


In [ ]:
session_action["id"] = session["id"] 

In [ ]:
session_action_sum = session_action.groupby("id").sum() #어떤 행동을 몇번씩 했는 지

In [ ]:
session["action"].nunique()

In [ ]:
all_data = pd.concat([train,test])
all_data

In [ ]:
all_data = pd.merge(all_data,id_count, on="id", how="left")
all_data = pd.merge(all_data,id_sec, on="id", how="left")
all_data = pd.merge(all_data,id_nunique, on="id", how="left")
all_data = pd.merge(all_data,session_action_sum, on='id',how="left")

In [ ]:
all_data

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(20,12))
# sns.boxplot(all_data["country_destination"],all_data["action_type_nunique"], showfliers=False)

In [ ]:
all_data.nunique()

In [ ]:
all_data.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20,12))
sns.countplot(all_data['first_affiliate_tracked'],hue = all_data['country_destination'])

In [ ]:
all_data["date_account_created"] = pd.to_datetime(all_data["date_account_created"])
all_data["account_year"] = all_data["date_account_created"].dt.year
all_data["account_month"] = all_data["date_account_created"].dt.month
all_data["account_day"] = all_data["date_account_created"].dt.day
all_data["account_weekday"] = all_data["date_account_created"].dt.weekday

In [ ]:
all_data["timestamp_first_active"] = all_data["timestamp_first_active"].astype("str")
all_data["first_active_year"] = all_data["timestamp_first_active"].apply(lambda x:x[:4])
all_data["first_active_month"] = all_data["timestamp_first_active"].apply(lambda x:x[4:6])
all_data["first_active_day"] = all_data["timestamp_first_active"].apply(lambda x:x[6:8])
all_data["first_active_hour"] = all_data["timestamp_first_active"].apply(lambda x:x[8:10])

In [ ]:
all_data2 = all_data.drop(["country_destination","date_first_booking","date_account_created","id","timestamp_first_active"],1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in all_data2.columns[all_data2.dtypes == object]:
    all_data2[i] = le.fit_transform(list(all_data2[i]))


In [ ]:
all_data2.nunique()

In [ ]:
all_data2 = all_data2.fillna(-1)

In [ ]:
all_data2["age"].isnull().sum()

In [ ]:
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train,y_valid = train_test_split(train2,train["country_destination"], test_size=0.2, random_state=42, stratify=train["country_destination"])

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(max_depth = 7,learning_rate=0.02, iterations = 10000 ,task_type="GPU", )
cat.fit(X_train,y_train, eval_set=(X_valid,y_valid), early_stopping_rounds=100) #100번이상 점수가 좋아지지 않으면 자동으로 종료

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(max_depth = 7,learning_rate=0.02, iterations = 2000 ,task_type="GPU" )
cat.fit(train2,train["country_destination"], early_stopping_rounds=100)
result = cat.predict_proba(test2)

In [ ]:
ids = []
country_list = []

In [ ]:
sub = pd.read_csv("/kaggle/input/airbnb-recruiting-new-user-bookings/sample_submission_NDF.csv.zip")

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['country_destination'])
for i in range(len(sub)):
    index = sub["id"][i]
    ids += [index] * 5
    country_list += le.inverse_transform(np.argsort(result[i])[::-1])[:5].tolist()

In [ ]:
len(ids)

In [ ]:
len(country_list)

In [ ]:
# pd.Series(rf.feature_importances_,index=all_data2["id"])

In [ ]:
sub = pd.DataFrame(np.column_stack((ids,country_list)),columns=["id","country"])
sub.to_csv("sub_airbnb.csv", index=False)

In [ ]:
plt.figure(figsize=(20,12))
sns.countplot(sub["country"])